### Semantic Chunking
- SemanticChunker is a document splitter that uses embedding similarity between sentences to decide chunk boundaries.

- It ensures that each chunk is semantically coherent and not cut off mid-thought like traditional character/token splitters.

In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

/Users/binod.kafle/Documents/personal/learning/rag/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
## Initialize the model
model=SentenceTransformer('all-MiniLM-L6-v2')

## Sample text
text="""
LangChain is a framework for building applications with LLMs.
Langchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.
You can create chains, agents, memory, and retrievers.
The Eiffel Tower is located in Paris.
France is a popular tourist destination.
"""

# Step 1: Split into sentences
sentences=[s.strip() for s in text.split("\n") if s.strip()]

# step 2: Embed each sentence
embeddings=model.encode(sentences)

# Step 3: Initialize parameters
threshold = 0.7  # control chunk tightness
chunks = []
current_chunk=[sentences[0]]

## Step 4: Semantic grouping based on a threshold
for i in range(1, len(sentences)):
    sim = cosine_similarity(
        [embeddings[i - 1]],
        [embeddings[i]]
    )[0][0]

    if sim>=threshold:
        current_chunk.append(sentences[i])
    else:
        chunks.append(" ".join(current_chunk))
        current_chunk=[sentences[i]]

# Append the last chunk
chunks.append(" ".join(current_chunk))

# Output the chunks
print("\n📌 Semantic Chunks:")
for idx, chunk in enumerate(chunks):
    print(f"\nChunk {idx+1}:\n{chunk}")


📌 Semantic Chunks:

Chunk 1:
LangChain is a framework for building applications with LLMs. Langchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.

Chunk 2:
You can create chains, agents, memory, and retrievers.

Chunk 3:
The Eiffel Tower is located in Paris.

Chunk 4:
France is a popular tourist destination.


### RAG Pipeline Modular Coding

In [14]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chat_models import init_chat_model
from langchain_classic.schema.runnable import RunnableLambda, RunnableMap
from langchain_classic.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

import os
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


In [11]:
# Custom semantic chunker with Threshold

class ThresholdSemanticChunker:
    def __init__(self, model_name="all-MiniLM-L6-v2", threshold=0.7):
        self.model=SentenceTransformer(model_name)
        self.threshold=threshold

    def split(self, text: str):
        sentences = [s.strip() for s in text.split('.') if s.strip()]
        embeddings = self.model.encode(sentences)
        chunks = []
        current_chunk = [sentences[0]]

        for i in range(1, len(sentences)):
            sim = cosine_similarity([embeddings[i - 1]], [embeddings[i]])[0][0]
            if sim >= self.threshold:
                current_chunk.append(sentences[i])
            else:
                chunks.append(". ".join(current_chunk) + ".")
                current_chunk = [sentences[i]]

        chunks.append(". ".join(current_chunk) + ".")
        return chunks

    def split_documents(self,docs):
        result=[]
        for doc in docs:
            for chunk in self.split(doc.page_content):
                result.append(Document(page_content=chunk, metadata=doc.metadata))

        return result

In [7]:
# Sample text
sample_text = """
LangChain is a framework for building applications with LLMs.
Langchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.
You can create chains, agents, memory, and retrievers.
The Eiffel Tower is located in Paris.
France is a popular tourist destination.
"""

doc = Document(page_content=sample_text)
doc

Document(metadata={}, page_content='\nLangChain is a framework for building applications with LLMs.\nLangchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.\nYou can create chains, agents, memory, and retrievers.\nThe Eiffel Tower is located in Paris.\nFrance is a popular tourist destination.\n')

In [12]:
# Chunking
chunker=ThresholdSemanticChunker(threshold=0.7)
chunks=chunker.split_documents([doc])
chunks

[Document(metadata={}, page_content='LangChain is a framework for building applications with LLMs. Langchain provides modular abstractions to combine LLMs with tools like OpenAI and Pinecone.'),
 Document(metadata={}, page_content='You can create chains, agents, memory, and retrievers.'),
 Document(metadata={}, page_content='The Eiffel Tower is located in Paris.'),
 Document(metadata={}, page_content='France is a popular tourist destination.')]

In [15]:
### VectorStore
embedding=HuggingFaceEmbeddings()
vectorstore=FAISS.from_documents(chunks,embedding)
retriever=vectorstore.as_retriever()

In [16]:
## Prompt Template

# --- 5. Prompt Template ---
template = """Answer the question based on the following context:

{context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based on the following context:\n\n{context}\n\nQuestion: {question}\n')

In [17]:
## LLM
llm=init_chat_model(model="groq:openai/gpt-oss-20b",temperature=0.4)

### LCEL Chain With retrieval

rag_chain=(
    RunnableMap(
        {
        "context": lambda x: retriever.invoke(x["question"]),
        "question": lambda x: x["question"],
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

# --- 8. Run Query ---
query = {"question": "What is LangChain used for?"}
result = rag_chain.invoke(query)

print(result)

LangChain is a framework for building applications that use large language models (LLMs). It offers modular abstractions that let developers combine LLMs with other tools (e.g., OpenAI, Pinecone) to create chains, agents, memory, and retrievers.


### Semantic chunker With Langchain

In [18]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_experimental.text_splitters import SemanticChunker

In [20]:
# Load the documents
loader = TextLoader("langchain_intro.txt")
docs = loader.load()

# Initialize embedding model
embedding = HuggingFaceEmbeddings(model="all-MiniLM-L6-v2")

# create semantic chunker
chunker = SemanticChunker(embedding)

## Result
for i,chunk in enumerate(chunks):
    print(f"\n chunk {i+1}:\n{chunk.page_content}")


NameError: name 'SemanticChunker' is not defined